In [43]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()

from tensorflow import feature_column
from tensorflow.keras import layers
# tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split

%matplotlib inline

ValueError: tf.enable_eager_execution must be called at program startup.

In [37]:
dataframe = pd.read_csv("doginfo.csv")
dataframe.head()

,age,kindCd,neuterYn,processState,sexCd,weight,noticeSdt,noticeEdt,noticeDays,age2
0,2008,푸들,N,0,F,7.46,2020-01-20,2020-01-30,10,12
1,2019,믹스견,N,1,M,7.00,2020-01-02,2020-01-16,14,1
2,2018,믹스견,U,0,M,4.50,2020-01-02,2020-01-13,11,2
3,2019,진도견,N,0,M,10.00,2020-01-02,2020-01-10,8,1
4,2016,믹스견,N,0,M,6.00,2020-01-02,2020-01-10,8,4


In [38]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), '훈련 샘플')
print(len(val), '검증 샘플')
print(len(test), '테스트 샘플')

14586 훈련 샘플
3647 검증 샘플
4559 테스트 샘플


In [39]:
# 판다스 데이터프레임으로부터 tf.data 데이터셋을 만들기 위한 함수
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('processState')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [40]:
batch_size = 5 # 예제를 위해 작은 배치 크기를 사용합니다.
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [41]:
for feature_batch, label_batch in train_ds.take(1):
  print('전체 특성:', list(feature_batch.keys()))
  print('나이 특성의 배치:', feature_batch['age'])
  print('타깃의 배치:', label_batch )


# 특성 열을 시험해 보기 위해 샘플 배치를 만듭니다.
example_batch = next(iter(train_ds))[0]

# 특성 열을 만들고 배치 데이터를 변환하는 함수
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [14]:
# 임베딩 열의 입력은 앞서 만든 범주형 열입니다.
kindCd_embedding = feature_column.embedding_column(kindCd, dimension=114)
demo(kindCd_embedding)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.